In [2]:
# useful notebook for sorting out talking to our databasae
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect
# and some other fine things leftover from the homework, really only need Pandas:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
# create an engine to project-2.db
engine = create_engine("sqlite:///project-2.db")

In [4]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
print('tables inside of our project-2.db')
print(inspector.get_table_names())
print('')

tables inside of our project-2.db
['gun_violence_data']



In [5]:
# let's see what's in those tables
print(' -- columns inside of gun_violence_data table')
columns = inspector.get_columns('gun_violence_data')
for column in columns:
    print(column["name"], column["type"])
print('')

 -- columns inside of gun_violence_data table
IncidentID INTEGER
Date TEXT
State TEXT
City_or_County TEXT
Address TEXT
Number_Killed INTEGER
Number_Injured INTEGER
Incident_URL TEXT
Source_URL TEXT
Incident_URL_Fields_Missing INTEGER
Congressional_District INTEGER
Gun_Stolen TEXT
Gun_Type TEXT
Incident_Characteristics TEXT
Latitude NUMERIC
Location_Description TEXT
Longitude NUMERIC
Number_of_Guns_Involved INTEGER
Notes TEXT
Participant_Age TEXT
Participant_Age_Group TEXT
Participant_Gender TEXT
Participant_Name TEXT
Participant_Relationship TEXT
Participant_Status TEXT
Participant_Type TEXT
Sources TEXT
State_House_District INTEGER
State_Senate_District INTEGER



In [6]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# View all of the classes that automap found
Base.classes.keys()

['gun_violence_data']

In [8]:
# Save references to our table
GunV = Base.classes.gun_violence_data

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [10]:
print(' -- Let\'s see what we have we have in our GunV table' )
print(session.query(GunV).statement)
print('')



 -- Let's see what we have we have in our GunV table
SELECT gun_violence_data."IncidentID", gun_violence_data."Date", gun_violence_data."State", gun_violence_data."City_or_County", gun_violence_data."Address", gun_violence_data."Number_Killed", gun_violence_data."Number_Injured", gun_violence_data."Incident_URL", gun_violence_data."Source_URL", gun_violence_data."Incident_URL_Fields_Missing", gun_violence_data."Congressional_District", gun_violence_data."Gun_Stolen", gun_violence_data."Gun_Type", gun_violence_data."Incident_Characteristics", gun_violence_data."Latitude", gun_violence_data."Location_Description", gun_violence_data."Longitude", gun_violence_data."Number_of_Guns_Involved", gun_violence_data."Notes", gun_violence_data."Participant_Age", gun_violence_data."Participant_Age_Group", gun_violence_data."Participant_Gender", gun_violence_data."Participant_Name", gun_violence_data."Participant_Relationship", gun_violence_data."Participant_Status", gun_violence_data."Participant_Ty

In [4]:
print ("========================")
print("the above cells show how we can connect to our sqlite database")
print("the cells below demonstrate how I pulled out the lat and lng coords" )
print ("========================")

the above cells show how we can connect to our sqlite database
the cells below demonstrate how I pulled out the lat and lng coords


In [11]:
# let's get some Latitude data
lats = pd.read_sql_query("SELECT Latitude from gun_violence_data", engine)
#print(type(lats['Latitude'].iloc[0]))
#lats

In [12]:
lngs = pd.read_sql_query("SELECT Longitude from gun_violence_data", engine)
#print(type(lngs['Longitude'].iloc[0]))
#lngs

In [18]:
lats_lngs_df = pd.merge(lats, lngs, left_index=True, right_index=True)
print ("we now have a single dataframe of lats and lngs")
lats_lngs_df

we now have a single dataframe of lats and lngs


Latitude Longitude
0       38.0507  -84.4829
1       38.4333  -84.3542
2       38.2501   -85.815
3       38.8847  -77.1672
4       34.6347  -98.4222
...         ...       ...
164469  34.0303  -81.0176
164470  43.0972  -87.9578
164471  39.7784  -104.838
164472   37.957  -101.781
164473  40.1732  -80.2434

[164474 rows x 2 columns]

In [5]:
print ("========================")
print("the next thing to do is to produce the data format to feed to our API")
print("for (lat, lng) in my heatmap, a 'list of lists' turned out to be the right thing" )
print ("========================")

the next thing to do is to produce the data format to feed to our API
for (lat, lng) in my heatmap, a 'list of lists' turned out to be the right thing


In [19]:
# convert to list of lists
lats_lngs_list = lats_lngs_df.values.tolist()
lats_lngs_list

[[38.0507, -84.4829],
 [38.4333, -84.3542],
 [38.2501, -85.815],
 [38.8847, -77.1672],
 [34.6347, -98.4222],
 [35.5711, -96.0152],
 [40.8179, -73.8628],
 [40.6804, -73.9167],
 [40.6838, -73.9075],
 [40.8396, -73.8758],
 [40.6155, -74.0816],
 [40.6715, -73.9476],
 [40.8623, -73.8959],
 [40.6505, -73.9573],
 [38.356, -81.644],
 [29.9768, -90.055],
 [32.4921, -93.7789],
 [36.8513, -82.2958],
 [35.2517, -97.495],
 [34.1259, -118.186],
 [38.119, -83.1214],
 [39.1452, -84.4743],
 [41.7035, -87.6259],
 [39.6077, -84.3014],
 [41.6824, -83.5364],
 [38.0534, -84.4808],
 [37.0698, -83.3789],
 [37.7953, -85.6381],
 [36.8508, -76.2859],
 [36.9881, -76.4185],
 [36.9875, -76.4097],
 [37.0959, -76.4894],
 [37.1422, -76.5355],
 [36.1459, -80.251],
 [36.0633, -79.7785],
 [32.8458, -80.0293],
 [32.8461, -79.9813],
 [32.8548, -79.9977],
 [32.7866, -83.7199],
 [33.4717, -86.8846],
 [34.7954, -86.6138],
 [34.2291, -91.9779],
 [37.5724, -77.4825],
 [35.3182, -78.7225],
 [35.7688, -78.6173],
 [34.9624, -77.96

In [6]:
print ("========================")
print("now that you've used the notebook to produce your data")
print("you move the relevant code to your flask-app.py and plug it in" )
print ("========================")

now that you've used the notebook to produce your data
you move the relevant code to your flask-app.py and plug it in
